# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
This is a classification problem. The goal of this problem is to output a binary response in the form of pass or fail. If the goal was to predict the average or score that a student would receive then the problem would be a regression problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [17]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
print(student_data.shape)

Student data read successfully!
(395, 31)


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [18]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = len(student_data.columns[:-1])

# TODO: Calculate passing students
p_counter = 0
f_counter = 0
counter = 0
for student in student_data.passed:
    if student == 'no':
        f_counter += 1
    else:
        p_counter += 1
n_passed = p_counter
# TODO: Calculate failing students
n_failed = f_counter

# TODO: Calculate graduation rate
grad_rate = (p_counter / float(n_students)) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [19]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [20]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)

### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [21]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split
# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train = None
X_test = None
y_train = None
y_test = None
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=10, stratify=y_all)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **
Stochastic gradient descent
What are the general applications of each model?

Stochastic gradient descent is an efficient approach that can be applied to both classification and regression problems.

What are their strengths and weaknesses?

The strengths of stochastic gradient descent are that it is efficient and the ease of implementation. Each iteration of the algorithm is run on a single training sample instead of the whole training set. 

The weaknesses of the algorithm is that it updates on a single training sample for each iteration. Stochastic gradient descent is also very sensitive to feature scaling. It can converge to a local optimum instead of the global optimum.

Given what you know about the data, why did you choose these models to be applied?

I choose stochastic gradient descent due to its computational efficieny and its ability to perform classification on a dataset.

Logistic regression
What are the general applications of each model?

Logistic regression is a linear model for classification. So it fits the problem.

What are their strengths and weaknesses?

Logistic regression creates a linear boundry which should be able obtain a high accuracy on this particular example. There are two loss functions that can be used when using the algorithm. L1 and L2 regularization with the latter being the selection in this case.

Given what you know about the data, why did you choose these models to be applied?
I choose logistic regression for this problem due to this problem being a classification problem.

Gaussian Naive Bayes

What are the general applications of each model?

Gaussian Navie Bayes is used for classification problems.

What are their strengths and weaknesses?

The application of the gaussina navie bayes algorithm is extremely fast. Navie Bayes is known as a decent classifier. Only requires a small amount of training data. Doesn't have to deal with the curse of dimensionality.

It is not to be taken seriously when used for estimation problems.

Given what you know about the data, why did you choose these models to be applied?

I choose Gaussian Navie Bayes because this is a classification problem and it is known as a decent classifier.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [22]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [27]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn import linear_model
sgd = linear_model.SGDClassifier(random_state = 10)
# from sklearn import model_B
logistic = linear_model.LogisticRegression(random_state = 10)
# from skearln import model_C
from sklearn.naive_bayes import GaussianNB

# TODO: Initialize the three models
clf_A = sgd
clf_B = logistic
clf_C = GaussianNB()

# TODO: Set up the training set sizes
#X_train y_train
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A,X_train_100,y_train_100,X_test,y_test)
print
train_predict(clf_A,X_train_200,y_train_200,X_test,y_test)
print
train_predict(clf_A,X_train_300,y_train_300,X_test,y_test)
print
train_predict(clf_B,X_train_100,y_train_100,X_test,y_test)
print
train_predict(clf_B,X_train_200,y_train_200,X_test,y_test)
print
train_predict(clf_B,X_train_300,y_train_300,X_test,y_test)
print
train_predict(clf_C,X_train_100,y_train_100,X_test,y_test)
print
train_predict(clf_C,X_train_200,y_train_200,X_test,y_test)
print
train_predict(clf_C,X_train_300,y_train_300,X_test,y_test)
print

Training a SGDClassifier using a training set size of 100. . .
Trained model in 0.0043 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 0.5155.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.4792.

Training a SGDClassifier using a training set size of 200. . .
Trained model in 0.0009 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.0000.

Training a SGDClassifier using a training set size of 300. . .
Trained model in 0.0026 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8250.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8129.

Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8873.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7519.

Training a LogisticRegression using a training set size of 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - SGD **  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0008        |     0.0003              |   0.5155        | 0.4792        |
| 200               |        0.0002          |      0.0002           |    0.0         |     0.0     |
| 300               |    0.0003                |  0.0002                 |     0.8250        |    0.8129      |

** Classifer 2 - Logistic Regression **  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0003            |    0.0002               |    0.8873      |        0.7519        |
| 200               |     0.0004             |          0.0005    |   0.8493             |      0.7556          |
| 300               |        0.0002          |       0.0001          |     0.8440        |     0.7826     |

** Classifer 3 - Gaussian NB **  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0002      |        0.0002          |  0.8467           |   0.7538          |
| 200               |   0.0003               | 0.0004                 |      0.8205      |   0.7576            |
| 300               |    0.0004              |     0.0002             |   0.8106       |  0.7786               |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
The model that I choose as the best is the logistic regression model. This model is choosen as the best model due to it having the best predictive accuarcy of the testing set, along with having reasonable prediction time. The prediction times of all the models were reasonable. However, the F1 score which is the number of correct positives or precision and the number of correct negatives or recall weighted varied between the models. Stochastic gradient descent obtained a decent score on the training set of size 100. However, increasing the size of training set caused a weird reaction the score dropped to a unreasonable value. Consequently the same event happened on the testing set. Logistic regression the score on the training set decreased with the size of the training set increasing. However, the score on the testing set increased with the size of the training set increasing. This showed that with increasing model complexity the generalization of the model on the testing set is better (only up to the point we can see which is 300 training points). The score the Gaussian Navie Bayes classifier on the training set continued to decrease with the increasing size of the training set. However, the score on the testing set varied, there was a decrease then an increase. So the score of Naive Bayes was taken as unstable. Thus logistic regression was choosen as the best model.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **
Linear models the target value or prediction value is predicted to be a linear combination of the input variables. The input variables are the features of the data. Such as age gender etc. The features are assigned weights. Feature x_0 is always assigned the value 1. Then using these features and weights that are assigned to the features we obtain the linear combination that is used for prediction. Given a training set of data we attempt to fit a line to the data that minimizes a cost function. In terms of linear regression the cost function is a squared error function. This is the error or the predicted minus the actual value squared.

Logistic regression is a linear model that outputs the probability that an input belongs to a class is modeled using a logistic function. Logistic regression also minimizes a cost function. The target value of logistic regression is the classification of the input variables. The model uses a descision boundry to separate the data points into two classes. The probabilities are then converted to there respective classes.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [28]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn import grid_search
from sklearn import linear_model
logistic = linear_model.LogisticRegression(random_state = 10)
from sklearn.metrics import f1_score
import sys
# TODO: Create the parameters list you wish to tune
parameters = {'C':(.1,.2,.3,.4,.5,1,1.5,2)}

# TODO: Initialize the classifier
clf = logistic

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# gridSearchCV(model, parameters, scoring=f1_scorer)
grid_obj = grid_search.GridSearchCV(logistic, parameters, scoring=f1_scorer)
grid_obj.fit(X_train, y_train)

# Select the best settings for classifier
best_clf = grid_obj.best_estimator_

# Fit the algorithm to the training data
print "Final Model: "
print best_clf, "\n"

# Test algorithm's performance
print "F1 score for training set: {}\n".format(predict_labels(best_clf, X_train, y_train))
print "F1 score for test set: {}\n".format(predict_labels(best_clf, X_test, y_test))

Final Model: 
LogisticRegression(C=0.4, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=10, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False) 

Made predictions in 0.0004 seconds.
F1 score for training set: 0.844243792325

Made predictions in 0.0002 seconds.
F1 score for test set: 0.797202797203



### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final scores for the tuned logistic regression model:
F1 score for training set: 0.844243792325
F1 score for test set: 0.797202797203

The scores compared to the model where the C parameter or the inverse of regularization strength was not tuned obtained a higher training set score. Smaller values of C represent stronger regularization. However, the testing set score obtained a lower score. 

So, the tuned model is able to generalize the testing set better than the untuned model. This shown with the increase in the test set score from 0.7826 to 0.797202797203.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.